# Checkpoint 1

Reminder: 

- You are being evaluated for compeletion and effort in this checkpoint. 
- Avoid manual labor / hard coding as much as possible, everything we've taught you so far are meant to simplify and automate your process.

We will be working with the same `states_edu.csv` that you should already be familiar with from the tutorial.

We investigated Grade 8 reading score in the tutorial. For this checkpoint, you are asked to investigate another test. Here's an overview:

* Choose a specific response variable to focus on
>Grade 4 Math, Grade 4 Reading, Grade 8 Math
* Pick or create features to use
>Will all the features be useful in predicting test score? Are some more important than others? Should you standardize, bin, or scale the data?
* Explore the data as it relates to that test
>Create at least 2 visualizations (graphs), each with a caption describing the graph and what it tells us about the data
* Create training and testing data
>Do you want to train on all the data? Only data from the last 10 years? Only Michigan data?
* Train a ML model to predict outcome 
>Define what you want to predict, and pick a model in sklearn to use (see sklearn <a href="https://scikit-learn.org/stable/modules/linear_model.html">regressors</a>.
* Summarize your findings
>Write a 1 paragraph summary of what you did and make a recommendation about if and how student performance can be predicted

Include comments throughout your code! Every cleanup and preprocessing task should be documented.

Of course, if you're finding this assignment interesting (and we really hope you do!), you are welcome to do more than the requirements! For example, you may want to see if expenditure affects 4th graders more than 8th graders. Maybe you want to look into the extended version of this dataset and see how factors like sex and race are involved. You can include all your work in this notebook when you turn it in -- just always make sure you explain what you did and interpret your results. Good luck!

<h2> Data Cleanup </h2>

Import `numpy`, `pandas`, and `matplotlib`.

(Feel free to import other libraries!)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt


Load in the "states_edu.csv" dataset and take a look at the head of the data

In [ ]:
df = pd.read_csv('../data/states_edu.csv')
df.head()

You should always familiarize yourself with what each column in the dataframe represents. Read about the states_edu dataset here: https://www.kaggle.com/noriuk/us-education-datasets-unification-project

Use this space to rename columns, deal with missing data, etc. _(optional)_

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
df['AVG_MATH_4_SCORE'] = imputer.fit_transform(df[['AVG_MATH_4_SCORE']])
df = df.dropna(subset=['AVG_MATH_4_SCORE'])
df = df.dropna(subset=['PRIMARY_KEY'])
df = df.dropna(subset=['STATE'])
df = df.dropna(subset=['YEAR'])

<h2>Exploratory Data Analysis (EDA) </h2>

Chosen one of Grade 4 Reading, Grade 4 Math, or Grade 8 Math to focus on: *ENTER YOUR CHOICE HERE*

How many years of data are logged in our dataset? 

In [ ]:
print(df["YEAR"].unique())

Let's compare Michigan to Ohio. Which state has the higher average across all years in the test you chose?

In [ ]:
michigan_mean = df[df['STATE'] == 'MICHIGAN']['AVG_MATH_4_SCORE'].mean()
ohio_mean = df[df['STATE'] == 'OHIO']['AVG_MATH_4_SCORE'].mean()

if michigan_mean > ohio_mean:
    answer = "Michigan has the higher average test score across all years."
elif ohio_mean > michigan_mean:
    answer = "Ohio has the higher average test score across all years."
else:
    answer = "Michigan and Ohio have the same average test score across all years."

michigan_mean, ohio_mean, answer

Find the average for your chosen test across all states in 2019

In [ ]:
df_2019 = df[df['YEAR'] == 2019]
average_scores_by_state = df_2019.groupby('STATE')['AVG_MATH_4_SCORE']
overall_average_2019 = df_2019['AVG_MATH_4_SCORE'].mean()

For each state, find a maximum value for your chosen test score

Refer to the `Grouping and Aggregating` section in Tutorial 0 if you are stuck.

In [ ]:
max_scores_by_state = df.groupby('STATE')['AVG_MATH_4_SCORE'].max().reset_index()

<h2> Feature Engineering </h2>

After exploring the data, you can choose to modify features that you would use to predict the performance of the students on your chosen response variable. 

You can also create your own features. For example, perhaps you figured that maybe a state's expenditure per student may affect their overall academic performance so you create a expenditure_per_student feature.

Use this space to modify or create features.

In [ ]:
#Given the total expenditure and enrollment (number of students), one can calculate the expenditure per student 
#It might provide insights into how much investment is made per student in each state.
df['expenditure_per_student'] = df['TOTAL_EXPENDITURE'] / df['ENROLL']

#If someone's interested in changes in student performance over time
#Calculate the difference in average test scores from one year to the next as a new feature.
df.sort_values(by=['STATE', 'YEAR'], inplace=True)
df['math_4_score_diff'] = df.groupby('STATE')['AVG_MATH_4_SCORE'].diff()
df['reading_4_score_diff'] = df.groupby('STATE')['AVG_READING_4_SCORE'].diff()
#The proportion of the total expenditure spent on instruction might be a good indicator 
#..of how resources are allocated towards direct education versus administration or other costs.

df['instructional_expenditure_proportion'] = df['INSTRUCTION_EXPENDITURE'] / df['TOTAL_EXPENDITURE']

Feature engineering justification: **<BRIEFLY DESCRIBE WHY YOU MADE THE CHANGES THAT YOU DID\>**

<h2>Visualization</h2>

Investigate the relationship between your chosen response variable and at least two predictors using visualizations. Write down your observations.

**Visualization 1**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.regplot(x='expenditure_per_student', y='AVG_MATH_4_SCORE', data=df, scatter_kws={'alpha':0.5})
plt.title('Average Math Score for 4th Graders vs. Expenditure Per Student')
plt.xlabel('Expenditure Per Student')
plt.ylabel('Average Math Score for 4th Graders')
plt.show()

**<CAPTION FOR VIZ 1>**! Run the code and it's above

**Visualization 2**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(x='instructional_expenditure_proportion', y='AVG_MATH_4_SCORE', data=df, scatter_kws={'alpha':0.5})
plt.title('Average Math Score for 4th Graders vs. Instructional Expenditure Proportion')
plt.xlabel('Instructional Expenditure Proportion')
plt.ylabel('Average Math Score for 4th Graders')
plt.show()

**<CAPTION FOR VIZ 2>**

<h2> Data Creation </h2>

_Use this space to create train/test data_

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['AVG_MATH_4_SCORE', 'PRIMARY_KEY', 'STATE', 'YEAR'], axis=1) 
y = df['AVG_MATH_4_SCORE']

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

<h2> Prediction </h2>

ML Models [Resource](https://medium.com/@vijaya.beeravalli/comparison-of-machine-learning-classification-models-for-credit-card-default-data-c3cf805c9a5a)

In [ ]:
# import your sklearn class here

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [ ]:
# create your model here

imputer = SimpleImputer(strategy='median')
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)
pipeline = make_pipeline(imputer, model)

pipeline.fit(X_train, y_train)


In [ ]:
imputer = SimpleImputer(strategy='median')
model = LogisticRegression()
model.fit(X_train, y_train)
pipeline = make_pipeline(imputer, model)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

## Evaluation

Choose some metrics to evaluate the performance of your model, some of them are mentioned in the tutorial.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

We have copied over the graphs that visualize the model's performance on the training and testing set. 

Change `col_name` and modify the call to `plt.ylabel()` to isolate how a single predictor affects the model.

In [ ]:
col_name = 'expenditure_per_student'

f = plt.figure(figsize=(12,6))
plt.scatter(X_train[col_name], y_train, color = "red")
plt.scatter(X_train[col_name], model.predict(X_train), color = "green")

plt.legend(['True Training','Predicted Training'])
plt.xlabel(col_name)
plt.ylabel('AVG_MATH_4_SCORE')
plt.title("Model Behavior On Training Set")

In [ ]:
col_name = 'expenditure_per_student'

f = plt.figure(figsize=(12,6))
plt.scatter(X_test[col_name], y_test, color = "blue")
plt.scatter(X_test[col_name], model.predict(X_test), color = "black")

plt.legend(['True testing','Predicted testing'])
plt.xlabel(col_name)
plt.ylabel('AVG_MATH_4_SCORE')
plt.title("Model Behavior on Testing Set")

<h2> Summary </h2>

**<WRITE A PARAGRAPH SUMMARIZING YOUR WORK, FINDINGS, AND THE PERFORMANCE OF YOUR MODEL\>**

In [ ]:
#In this project, we analyzed a dataset to predict 4th graders' average math scores using features like 'expenditure_per_student' and 'instructional_expenditure_proportion'. After feature engineering and exploratory data analysis, we used a Logistic Regression model for prediction. The model was trained and tested, and its performance was evaluated through visualizations comparing actual and predicted scores. These visualizations helped assess how well the model captured the relationship between financial investment in education and student performance, providing valuable insights into the impact of educational expenditures on student outcomes.